In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.keys import Keys
import bs4
import time
import re

In [6]:
options = Options()
options.binary_location = r'C:\Program Files\Mozilla Firefox\firefox.exe'
driver = webdriver.Firefox(executable_path=r'C:\Users\Acer\scrapy_shopee_test\geckodriver.exe', options=options)
driver.get('https://shopee.co.th/')
time.sleep(5)
thai_button = driver.find_element_by_xpath('/html/body/div[2]/div[1]/div[1]/div/div[3]/div[1]/button')
thai_button.click()
time.sleep(3)
close_adver = driver.execute_script('return document.querySelector("shopee-banner-popup-stateful").shadowRoot.querySelector("div.shopee-popup__close-btn")')
close_adver.click()
time.sleep(3)
search = driver.find_element_by_xpath('/html/body/div[1]/div/header/div[2]/div/div[1]/div[1]/div/form/input')
search.send_keys('กาแฟพร้อมดื่ม') #ใส่เนื้อหาที่ต้องการที่จะ Scrape จากเว็บ Shopee
search.send_keys(Keys.ENTER)

driver.execute_script("document.body.style.MozTransform='scale(0.1)';")
driver.execute_script('document.body.style.MozTransformOrigin = "0 0";')

records=[]
for i in range(5):
    data = driver.page_source
    soup = bs4.BeautifulSoup(data)
    All_product=soup.select(".row>div")
    for Product in All_product:
        name = Product.select_one("div.col-xs-2-4 > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1)").text.strip()
        try:
            p = Product.select_one("div.col-xs-2-4 > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(2) > div:nth-child(2)").text.strip()
            if p == '':
                print(1/0)
        except:
            p = Product.select_one("div.col-xs-2-4 > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(2)").text.strip()
        p = p.replace(",", "").replace("฿", "").replace(" ", "")
        try:
            start_price = float(p)
        except:
            price = p.rpartition('-')[0]
            start_price = float(price)

        sales = Product.select_one("div.col-xs-2-4 > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(3) > div:nth-child(2)").text.strip().replace("พัน","000").replace("ล้าน","000000")
        try:
            sales = float(re.sub('\D','',sales))
        except:
            sales = 0
        province = Product.select_one("div.col-xs-2-4 > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(4)").text.strip()
        #print(name, p)
        records.append([name, start_price,sales,province])
        
    time.sleep(5)
    next_button = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/div/div[2]/div/div[3]/div/button[8]')
    next_button.click()
    time.sleep(5)
    
df = pd.DataFrame(records, columns=['Product','price','sales','province'])
df = df.set_index('Product')
df

,price,sales,province
Product,,,
Lamoon Cold Brew Concentrate 200 ml. กาแฟสกัดเย็นดอยช้าง(ชนิดเข้มข้น),75.0,18000.0,จังหวัดเชียงใหม่
Ajinomoto อายิโนะโมะโต๊ะผงชูรส 500กรัม,57.0,23000.0,จังหวัดสมุทรปราการ
Birdy Robusta 180 ml. x 6 Cans Birdy เบอร์ดี้ โรบัสต้า กาแฟปรุงสำเร็จพร้อมดื่ม โรบัสต้า 180 มล. แพค 6 กระป๋อง,90.0,579000.0,จังหวัดสมุทรปราการ
Birdy Robusta Low Sugar 180 ml. X 30 cans เบอร์ดี้ โรบัสต้า สูตรน้ำตาลน้อย 180 มล. x 30 กระป๋อง,385.0,544.0,จังหวัดสมุทรปราการ
Starbucks/COSTA/GIMOKA/KIMBO กาแฟ สตาร์บัคส์ บดสำเร็จพร้อมชง ผงกาแฟ เมล็ดกาแฟ 200g/200g/250g/200g,59.0,4000.0,จังหวัดเชียงใหม่
...,...,...,...
[+CODE11.15]📌ส่งฟรี+แถมแก้ว📌เมล็ดกาแฟดอยช้าง คั่วเข้ม/คั่วกลาง/คั่วอ่อน 1 กก.(250ก.×4ถุง) แถมแก้ว Doi Chang Professional,629.0,67000.0,จังหวัดกรุงเทพมหานคร
กาแฟทรีเมจิก 5in1ไม่มีน้ำตาล ขนาด 1 ห่อ 50 ซอง (1),219.0,47000.0,จังหวัดเชียงราย
"PK French Roast ขนาด 1,000g. เมล็ดกาแฟคั่วเข้มมาก ”ดอยปางขอน” คั่วสะใจ + ส่งไว คั่วใหม่ ขายปลีก-ส่ง ราคาชาวไร่",313.0,52000.0,จังหวัดเชียงราย


In [7]:
df.to_excel('week5_real.xlsx', encoding='utf-8')